In [2]:
# import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

import pandas as pd
import numpy as np
import datetime as dt

import sqlite3
import sqlalchemy as db
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify, render_template, request

import warnings
warnings.filterwarnings('ignore')

In [3]:
# sqlite3 nift
con = sqlite3.connect("movie_ratings_db.sqlite")
links_df = pd.read_sql_query("SELECT * FROM Links", con)
movies_df = pd.read_sql_query("SELECT * FROM Movies", con)
ratings_df = pd.read_sql_query("SELECT * FROM Ratings", con)
tags_df = pd.read_sql_query("SELECT * FROM Tags", con)

In [5]:
tags_df = tags_df.drop(columns='timestamp')
ratings_df = ratings_df.drop(columns='timestamp')

In [16]:
# need unique identifiers for each user movie pair, will delete later
tags_df['movieId'] = tags_df['movieId'].astype(str)
tags_df['userId'] = tags_df['userId'].astype(str)
tags_df['user_movie'] = tags_df['userId'].str.cat(tags_df['movieId'], sep='_')

# same story for ratings
ratings_df['movieId'] = ratings_df['movieId'].astype(str)
ratings_df['userId'] = ratings_df['userId'].astype(str)
ratings_df['user_movie'] = ratings_df['userId'].str.cat(ratings_df['movieId'], sep='_')

In [26]:
# merge on user_movie, our unique identifier
tags_ratings_df = pd.merge(tags_df, ratings_df, on='user_movie', how='left')
tags_ratings_df = tags_ratings_df.drop(columns=['userId_y','movieId_y','user_movie'])
# renaming columns with danger
tags_ratings_df.columns = ['userId','movieId','tag','rating']
tags_ratings_df =  tags_ratings_df.dropna(subset=['tag','rating'])
# leaving genre out for now

,userId,movieId,tag,rating
0,2,60756,funny,5.0
1,2,60756,Highly quotable,5.0
2,2,60756,will ferrell,5.0
3,2,89774,Boxing story,5.0
4,2,89774,MMA,5.0
...,...,...,...,...
3677,606,6107,World War II,4.0
3678,606,7382,for katie,4.5
3680,610,3265,gun fu,5.0
3681,610,3265,heroic bloodshed,5.0


In [ ]:
# creating numeric tag Ids

In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# scale ratings within 0 to 1 range
tags_ratings_df['scaled_rating'] = scaler.fit_transform(tags_ratings_df)[['rating']]
# pivot_table
user_tag_preferences = tags_ratings_df.pivot_table(index='userId', columns='tags', values='scaled_rating', fill_value=0)

ValueError: could not convert string to float: 'funny'